
# 🔄 Audio Preprocessing Pipeline for Federated Learning
This notebook processes `.wav` audio files into fixed-length `.npy` files using Mel-spectrograms. It supports both IID and non-IID client setups for federated learning experiments.

---

### 📌 Configuration


In [ ]:
import os
import shutil
from typing import Dict
from pathlib import Path
import numpy as np
import librosa
import matplotlib.pyplot as plt

# === CONFIGURATION ===
CONFIG = {
    "sampling_rate": 22050,
    "n_mels": 128,
    "fmax": 8000,
    "pad_or_truncate_len": 128 * 32,
    "valid_extensions": [".wav"],
    "labels": ["good", "bad"]
}

PREPROCESS_CONFIG = {
    "client_input_root": "../../resources/material/train-data/federated/{mode}",
    "client_output_root": "../../resources/material/train-data/federated/{mode}-npy"
}

### 🧼 Clean/Create Output Directory

In [41]:
def clean_and_create_dir(path: str):
    if os.path.exists(path):
        print(f"🧹 Removing existing output directory: {path}")
        shutil.rmtree(path)
    print(f"📁 Creating output directory: {path}")
    os.makedirs(path, exist_ok=True)


### 🔉 Feature Extraction Utilities

In [42]:
def extract_melspectrogram(y: np.ndarray, sr: int) -> np.ndarray:
    mel = librosa.feature.melspectrogram(
        y=y,
        sr=sr,
        n_mels=CONFIG["n_mels"],
        fmax=CONFIG["fmax"]
    )
    mel_db = librosa.power_to_db(mel, ref=np.max)
    mel_db_flat = mel_db.flatten()
    return pad_or_truncate(mel_db_flat)

def pad_or_truncate(vector: np.ndarray) -> np.ndarray:
    target_len = CONFIG["pad_or_truncate_len"]
    current_len = len(vector)
    if current_len > target_len:
        return vector[:target_len]
    elif current_len < target_len:
        return np.pad(vector, (0, target_len - current_len))
    return vector

def process_audio_file(filepath: str) -> np.ndarray:
    y, sr = librosa.load(filepath, sr=CONFIG["sampling_rate"])
    return extract_melspectrogram(y, sr)


### 🗂️ Client Audio Preprocessing

In [43]:
def convert_class_wavs_to_npy(input_dir: str, output_dir: str, label: str) -> None:
    input_class_dir = os.path.join(input_dir, label)
    output_class_dir = os.path.join(output_dir, label)
    os.makedirs(output_class_dir, exist_ok=True)

    for fname in os.listdir(input_class_dir):
        if any(fname.endswith(ext) for ext in CONFIG["valid_extensions"]):
            wav_path = os.path.join(input_class_dir, fname)
            npy_path = os.path.join(output_class_dir, fname.replace(".wav", ".npy"))
            try:
                mel_features = process_audio_file(wav_path)
                print(f"📏 {label.upper()} - {fname}: feature length = {mel_features.shape[0]}")
                np.save(npy_path, mel_features)
                print(f"✅ Saved: {npy_path}")
            except Exception as e:
                print(f"❌ Failed to process {wav_path}: {e}")

def preprocess_client_audio(input_dir, output_dir, config):
    """
    Process all .wav files under good/bad labels in input_dir and save .npy files in output_dir.
    
    Args:
        input_dir (str): Path to client folder containing good/ and bad/ folders.
        output_dir (str): Path to save extracted features as .npy files.
        config (dict): Global config for audio preprocessing.
    """
    for label in config["labels"]:
        input_label_dir = os.path.join(input_dir, label)
        output_label_dir = os.path.join(output_dir, label)

        if not os.path.exists(input_label_dir):
            print(f"⚠️  Label folder missing: {input_label_dir}")
            continue

        os.makedirs(output_label_dir, exist_ok=True)
        file_list = [
            f for f in os.listdir(input_label_dir)
            if os.path.splitext(f)[1].lower() in config["valid_extensions"]
        ]

        print(f"📦 {label.upper()} files: {len(file_list)}")
        
        for fname in file_list:
            wav_path = os.path.join(input_label_dir, fname)
            mel = process_audio_file(wav_path)
            out_path = os.path.join(output_label_dir, fname.replace(".wav", ".npy"))
            np.save(out_path, mel)



### 🧪 Flat Test Client Preprocessing

In [44]:
def convert_test_client(input_dir, output_dir, config):
    """
    Converts all .wav files in flat test client dir into .npy and saves them in flat output dir.
    
    Args:
        input_dir (str): Path to flat test client WAVs.
        output_dir (str): Output path for .npy features.
        config (dict): Global config for preprocessing.
    """
    os.makedirs(output_dir, exist_ok=True)
    files = [
        f for f in os.listdir(input_dir)
        if os.path.splitext(f)[1].lower() in config["valid_extensions"]
    ]

    print(f"🧪 Found {len(files)} files in test client folder")

    for fname in files:
        wav_path = os.path.join(input_dir, fname)
        mel = process_audio_file(wav_path)
        out_path = os.path.join(output_dir, fname.replace(".wav", ".npy"))
        np.save(out_path, mel)


### 🔁 Run Preprocessing for All Clients

In [ ]:
def process_all_clients(base_config: dict, mode: str):
    print(f"\n🚀 Starting federated preprocessing for mode: {mode}")

    # Resolve mode-specific paths from the base config
    input_root = base_config["client_input_root"].format(mode=mode)
    output_root = base_config["client_output_root"].format(mode=mode)
    test_input_dir = os.path.join(input_root, "test_clients")
    test_output_dir = os.path.join(output_root, "test_clients")

    print(f"📂 Input Directory  : {input_root}")
    print(f"📂 Output Directory : {output_root}")

    # Clean and create npy dir
    clean_and_create_dir(output_root)
    
    # Prepare output folder
    if os.path.exists(output_root):
        print(f"🧹 Removing existing output directory: {output_root}")
        shutil.rmtree(output_root)

    print(f"📁 Creating output directory: {output_root}")
    os.makedirs(output_root, exist_ok=True)

    # Construct an expanded config with resolved paths and mode
    expanded_config = {
        **CONFIG,
        "mode": mode,
        "input_root": input_root,
        "output_root": output_root
    }

    # Process each client folder
    for client in sorted(os.listdir(input_root)):
        if client == "test_clients":
            continue
        input_client_path = os.path.join(input_root, client)
        output_client_path = os.path.join(output_root, client)

        if os.path.isdir(input_client_path):
            print(f"\n🎧 Processing client: {client}")
            preprocess_client_audio(input_client_path, output_client_path, expanded_config)

    # Process flat test_clients folder
    if os.path.exists(test_input_dir):
        print(f"\n🧪 Processing flat test_clients from: {test_input_dir}")
        convert_test_client(test_input_dir, test_output_dir, expanded_config)
    else:
        print(f"⚠️  No flat test_clients found at: {test_input_dir}")


▶️ Main Entry Point IID

In [ ]:
# Process all IID clients
process_all_clients(PREPROCESS_CONFIG, "IID")


🚀 Starting federated preprocessing for mode: IID
📂 Input Directory  : ../../resources/material/train-data/federated/IID
📂 Output Directory : ../../resources/material/train-data/federated/IID-npy
🧹 Removing existing output directory: ../../resources/material/train-data/federated/IID-npy
📁 Creating output directory: ../../resources/material/train-data/federated/IID-npy
🧹 Removing existing output directory: ../../resources/material/train-data/federated/IID-npy
📁 Creating output directory: ../../resources/material/train-data/federated/IID-npy

🎧 Processing client: client_1
📦 GOOD files: 2185
📦 BAD files: 2185

🎧 Processing client: client_2
📦 GOOD files: 2185
📦 BAD files: 2185

🎧 Processing client: client_3
📦 GOOD files: 2185
📦 BAD files: 2185

🎧 Processing client: client_4
📦 GOOD files: 2185
📦 BAD files: 2185

🎧 Processing client: client_5
📦 GOOD files: 2185
📦 BAD files: 2185

🧪 Processing flat test_clients from: ../../resources/material/train-data/federated/IID/test_clients
🧪 Found 200 f

▶️ Main Entry Point non-IID

In [47]:
# Process all non-IID clients
process_all_clients(PREPROCESS_CONFIG, "non-IID")


🚀 Starting federated preprocessing for mode: non-IID
📂 Input Directory  : ../../resources/material/train-data/federated/non-IID
📂 Output Directory : ../../resources/material/train-data/federated/non-IID-npy
📁 Creating output directory: ../../resources/material/train-data/federated/non-IID-npy
🧹 Removing existing output directory: ../../resources/material/train-data/federated/non-IID-npy
📁 Creating output directory: ../../resources/material/train-data/federated/non-IID-npy

🎧 Processing client: client_1
📦 GOOD files: 4915
📦 BAD files: 1092

🎧 Processing client: client_2
📦 GOOD files: 4369
📦 BAD files: 2184

🎧 Processing client: client_3
📦 GOOD files: 2733
📦 BAD files: 3277

🎧 Processing client: client_4
📦 GOOD files: 2
📦 BAD files: 4369

🎧 Processing client: client_5
📦 GOOD files: 0
📦 BAD files: 2781

🧪 Processing flat test_clients from: ../../resources/material/train-data/federated/non-IID/test_clients
🧪 Found 200 files in test client folder
